In [1]:
import os

import numpy
import pandas as pd
from pyspark.sql.functions import col, lit

from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

In [2]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

# Create SparkSession

In [3]:
spark = SparkSession(sc)

# Read NBA data

In [4]:
# dfs = []
# source_dir = 'BasketballStats/data'
# column_names = ['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'yr']
# beg, end = [1970,2016]
# for i in range(beg,end+1):
#     _df = pd.read_csv(os.path.join(source_dir,'leagues_NBA_{}_per_game_per_game.csv'.format(i)))
#     _df['yr'] = i
#     _df.columns = column_names
#     dfs.append(_df)
# df_all = pd.concat(dfs, ignore_index=True)

# df_all = df_all[df_all.Age != 'Age']
# df_all = df_all[df_all.PF != 'PF']
# df_all.to_csv('all_data.csv')

In [12]:
# saved above file to hdfs
df = spark.read.option('header','true').csv("/BasketballStats/all_data.csv").filter('FG != "FG"')

# Cleansing

In [30]:
print(df.head())
print(df.count())
print(df.schema.names)
print(df.dtypes)

Row(_c0='0', Rk='1', Player='Zaid Abdul-Aziz', Pos='PF', Age='23', Tm='MIL', G='80', GS=None, MP='20.5', FG='3.0', FGA='6.8', FG%='.434', 3P=None, 3PA=None, 3P%=None, 2P='3.0', 2PA='6.8', 2P%='.434', eFG%='.434', FT='1.5', FTA='2.3', FT%='.643', ORB=None, DRB=None, TRB='7.5', AST='0.8', STL=None, BLK=None, TOV=None, PF='2.1', PTS='7.4', yr='1970')
17534
['_c0', 'Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'yr']
[('_c0', 'string'), ('Rk', 'string'), ('Player', 'string'), ('Pos', 'string'), ('Age', 'string'), ('Tm', 'string'), ('G', 'string'), ('GS', 'string'), ('MP', 'string'), ('FG', 'string'), ('FGA', 'string'), ('FG%', 'string'), ('3P', 'string'), ('3PA', 'string'), ('3P%', 'string'), ('2P', 'string'), ('2PA', 'string'), ('2P%', 'string'), ('eFG%', 'string'), ('FT', 'string'), ('FTA', 'string'), ('FT%', 'string'), ('ORB', 'strin

In [33]:
a = 1 if 1 == 1 else 0

In [36]:
dtype_dict = {'Player' : StringType, 'Pos' : StringType, 'Tm' : StringType, 'Age' : IntegerType, 'G' : IntegerType, 'GS' : IntegerType, 'yr' : IntegerType}

df2 = df.fillna('0')

for c in df2.schema.names[6:]:
    dtype = DoubleType if c not in dtype_dict.keys() else dtype_dict[c]
    df2 = df2.withColumn(c,df2[c].cast(dtype()))

[('_c0', 'string'),
 ('Rk', 'string'),
 ('Player', 'string'),
 ('Pos', 'string'),
 ('Age', 'string'),
 ('Tm', 'string'),
 ('G', 'int'),
 ('GS', 'int'),
 ('MP', 'double'),
 ('FG', 'double'),
 ('FGA', 'double'),
 ('FG%', 'double'),
 ('3P', 'double'),
 ('3PA', 'double'),
 ('3P%', 'double'),
 ('2P', 'double'),
 ('2PA', 'double'),
 ('2P%', 'double'),
 ('eFG%', 'double'),
 ('FT', 'double'),
 ('FTA', 'double'),
 ('FT%', 'double'),
 ('ORB', 'double'),
 ('DRB', 'double'),
 ('TRB', 'double'),
 ('AST', 'double'),
 ('STL', 'double'),
 ('BLK', 'double'),
 ('TOV', 'double'),
 ('PF', 'double'),
 ('PTS', 'double'),
 ('yr', 'int')]

In [38]:
df2.head()

Row(_c0='0', Rk='1', Player='Zaid Abdul-Aziz', Pos='PF', Age='23', Tm='MIL', G=80, GS=0, MP=20.5, FG=3.0, FGA=6.8, FG%=0.434, 3P=0.0, 3PA=0.0, 3P%=0.0, 2P=3.0, 2PA=6.8, 2P%=0.434, eFG%=0.434, FT=1.5, FTA=2.3, FT%=0.643, ORB=0.0, DRB=0.0, TRB=7.5, AST=0.8, STL=0.0, BLK=0.0, TOV=0.0, PF=2.1, PTS=7.4, yr=1970)

In [31]:
from pyspark.sql.types import DoubleType, IntegerType, StringType

df3 = df2.withColumn("GS", df2["GS"].cast(DoubleType()))
df3.dtypes

[('_c0', 'string'),
 ('Rk', 'string'),
 ('Player', 'string'),
 ('Pos', 'string'),
 ('Age', 'string'),
 ('Tm', 'string'),
 ('G', 'string'),
 ('GS', 'double'),
 ('MP', 'string'),
 ('FG', 'string'),
 ('FGA', 'string'),
 ('FG%', 'string'),
 ('3P', 'string'),
 ('3PA', 'string'),
 ('3P%', 'string'),
 ('2P', 'string'),
 ('2PA', 'string'),
 ('2P%', 'string'),
 ('eFG%', 'string'),
 ('FT', 'string'),
 ('FTA', 'string'),
 ('FT%', 'string'),
 ('ORB', 'string'),
 ('DRB', 'string'),
 ('TRB', 'string'),
 ('AST', 'string'),
 ('STL', 'string'),
 ('BLK', 'string'),
 ('TOV', 'string'),
 ('PF', 'string'),
 ('PTS', 'string'),
 ('yr', 'string')]

# Summary stats
* number distinct players
* most points per game

In [89]:
stats_df.groupBy('Age').count().show()

+---+-----+
|Age|count|
+---+-----+
| 29| 1126|
| 42|    3|
| 30| 1010|
| 34|  398|
| 22| 1392|
| 28| 1261|
| 35|  272|
| 43|    1|
| 31|  841|
| 18|   13|
| 27| 1448|
| 26| 1602|
|Age|  846|
| 19|   97|
| 23| 1980|
| 41|    5|
| 38|   75|
| 40|   15|
| 25| 1726|
| 44|    1|
+---+-----+
only showing top 20 rows



In [70]:
stats_df.select('Player')

3200

In [82]:
spark.sql('SELECT * FROM nba_stats limit 10').take(10)

[Row(Rk='1', Player='Quincy Acy', Pos='PF', Age='25', Tm='SAC', G='59', GS='29', MP='14.8', FG='2.0', FGA='3.6', FG%='.556', 3P='0.3', 3PA='0.8', 3P%='.388', 2P='1.7', 2PA='2.8', 2P%='.606', eFG%='.600', FT='0.8', FTA='1.2', FT%='.735', ORB='1.1', DRB='2.1', TRB='3.2', AST='0.5', STL='0.5', BLK='0.4', TOV='0.5', PF='1.7', PTS='5.2'),
 Row(Rk='2', Player='Jordan Adams', Pos='SG', Age='21', Tm='MEM', G='2', GS='0', MP='7.5', FG='1.0', FGA='3.0', FG%='.333', 3P='0.0', 3PA='0.5', 3P%='.000', 2P='1.0', 2PA='2.5', 2P%='.400', eFG%='.333', FT='1.5', FTA='2.5', FT%='.600', ORB='0.0', DRB='1.0', TRB='1.0', AST='1.5', STL='1.5', BLK='0.0', TOV='1.0', PF='1.0', PTS='3.5'),
 Row(Rk='3', Player='Steven Adams', Pos='C', Age='22', Tm='OKC', G='80', GS='80', MP='25.2', FG='3.3', FGA='5.3', FG%='.613', 3P='0.0', 3PA='0.0', 3P%=None, 2P='3.3', 2PA='5.3', 2P%='.613', eFG%='.613', FT='1.4', FTA='2.5', FT%='.582', ORB='2.7', DRB='3.9', TRB='6.7', AST='0.8', STL='0.5', BLK='1.1', TOV='1.1', PF='2.8', PTS='8

# Notes

Error: 

```hive> create table employee(emp_id int,name string,salary double);
FAILED: Execution Error, return code 1 from org.apache.hadoop.hive.ql.exec.DDLTask. java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.metastore.HiveMetaStoreClient```

rm   metastore_db/*.lck

In [63]:
def update_columns(data,new_columns):
    """http://stackoverflow.com/questions/34077353/how-to-change-dataframe-column-names-in-pyspark"""
    orig_columns = data.schema.names
    for i,nc in enumerate(new_columns):
        _df.select(_df[i].alias())
    print(_df.schema.names)
#     old_columns = data.schema.names
#     print(new_columns)
#     return reduce(lambda data, idx: data.withColumnRenamed(old_columns[idx], new_columns[idx]), range(len(old_columns)), data)

dfs = []
beg, end = [1970,2016]
column_names = ['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'yr']
# beg, end = [1970,1975]
for i in range(beg,end+1):
    _df = spark.read.option('header','true').csv("/BasketballStats/data/*{}*.csv".format(i)).filter('FG != "FG"')
    _df = _df.withColumn('yr',lit(i))
    _df = update_columns(_df,column_names)
    dfs.append(_df)
    
stats_df = unionAll(*dfs)

Py4JJavaError: An error occurred while calling o4492.csv.
: java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
	at org.apache.hadoop.hive.ql.session.SessionState.start(SessionState.java:522)
	at org.apache.spark.sql.hive.client.HiveClientImpl.<init>(HiveClientImpl.scala:189)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.spark.sql.hive.client.IsolatedClientLoader.createClient(IsolatedClientLoader.scala:258)
	at org.apache.spark.sql.hive.HiveUtils$.newClientForMetadata(HiveUtils.scala:359)
	at org.apache.spark.sql.hive.HiveUtils$.newClientForMetadata(HiveUtils.scala:263)
	at org.apache.spark.s